# 🧠 Model Training

This notebook is designed to be deployed into a containerized node.

## 1. Python Setup

Run in the terminal:
```
python -m venv ultralytics-env
. /ultralytics-env/bin/activate
pip install -r dock_requirements.txt
```

In [2]:
from ultralytics import YOLO
import gdown
from dotenv import load_dotenv
import os
from zipfile import ZipFile
import pandas as pd
import numpy as np
import shutil

## 2. Download Data Files

In [2]:
# Create a folder to download zip files to
os.makedirs(os.path.join('data', 'downloads'), exist_ok=True)
# Create the processed data folder
os.makedirs(os.path.join('data', 'processed'), exist_ok=True)

In [3]:
# Load environment variables from .env file
load_dotenv('secrets.env')

True

In [4]:
# Set download variables
BASELINE_DATA_ID = os.getenv('BASELINE')
MODEL_3D_DATA_ID = os.getenv('3D_MODELS')
CLIPART_DATA_ID = os.getenv('CLIPART')
GENAI_DATA_ID = os.getenv('GENAI')
TEST_DATA_ID = os.getenv('TEST_DATA')

In [5]:
# Downloads base URL
BASE_URL = "https://drive.google.com/file/d/"
URL_SUFFIX = "/view?usp=sharing"
# Download path
DOWNLOAD_PATH = os.path.join('data', 'downloads')

If downloads aren't working it may be necessary to download your cookies for google drive and placing it in the `~/.cache/gdown/` folder.

```
Place cookies file in working directory
Use: mv cookies.txt ~/.cache/gdown/cookies.txt
Retry download
```

In [ ]:
# Download the test data set
gdown.download(f"{BASE_URL}{TEST_DATA_ID}{URL_SUFFIX}", os.path.join(DOWNLOAD_PATH, 'test_data.zip'), quiet=False, fuzzy=True)

Downloading...
From (original): https://drive.google.com/uc?id=1YrQL8xF59OqhaM-xssrwEOTJi8JIM759
From (redirected): https://drive.google.com/uc?id=1YrQL8xF59OqhaM-xssrwEOTJi8JIM759&confirm=t&uuid=daa3c98f-35ed-4ab5-a027-d78794ba2230
To: /home/developer/synth-data-proj/data/downloads/test_data.zip
100%|██████████| 2.43G/2.43G [01:02<00:00, 38.9MB/s]


'data/downloads/test_data.zip'

In [8]:
# Download the baseline data set
gdown.download(f"{BASE_URL}{BASELINE_DATA_ID}{URL_SUFFIX}", os.path.join(DOWNLOAD_PATH, 'baseline_data.zip'), quiet=False, fuzzy=True)

Downloading...
From (original): https://drive.google.com/uc?id=1-3U-WfRgL3krWNlIoZOJ4qipGywepOxg
From (redirected): https://drive.google.com/uc?id=1-3U-WfRgL3krWNlIoZOJ4qipGywepOxg&confirm=t&uuid=2e57abab-b814-4342-bd81-6348038130c6
To: /home/developer/synth-data-proj/data/downloads/baseline_data.zip
100%|██████████| 26.1G/26.1G [10:15<00:00, 42.4MB/s]


'data/downloads/baseline_data.zip'

In [45]:
# Download the 3D model data set
gdown.download(f"{BASE_URL}{MODEL_3D_DATA_ID}{URL_SUFFIX}", os.path.join(DOWNLOAD_PATH, '3D_model_data.zip'), quiet=False, fuzzy=True)

Downloading...
From (original): https://drive.google.com/uc?id=1kHtr0K5ImHuZwEi_xktJQOGbD4x2PdEg
From (redirected): https://drive.google.com/uc?id=1kHtr0K5ImHuZwEi_xktJQOGbD4x2PdEg&confirm=t&uuid=8c96948a-7534-4952-8391-dccc05816ccf&at=ABOpFIrBEKF5l7XU83KIb0_D_0tm%3A1761135664969
To: /home/developer/synth-data-proj/data/downloads/3D_model_data.zip
100%|██████████| 42.5G/42.5G [17:23<00:00, 40.8MB/s]  


'data/downloads/3D_model_data.zip'

In [51]:
# Download the clipart data set
gdown.download(f"{BASE_URL}{CLIPART_DATA_ID}{URL_SUFFIX}", os.path.join(DOWNLOAD_PATH, 'clipart_data.zip'), quiet=False, fuzzy=True)

Downloading...
From (original): https://drive.google.com/uc?id=1f7yEK6WCaN_Af-opQ8c6Ey1JRQ-meMzE
From (redirected): https://drive.google.com/uc?id=1f7yEK6WCaN_Af-opQ8c6Ey1JRQ-meMzE&confirm=t&uuid=24c8ea96-7974-461a-8031-4290f2cb442a&at=ABOpFIqAYulz-cFA2smLaKAOXZOO%3A1761137657545
To: /home/developer/synth-data-proj/data/downloads/clipart_data.zip
100%|██████████| 42.3G/42.3G [22:47<00:00, 30.9MB/s]  


'data/downloads/clipart_data.zip'

In [11]:
# Download the genai data set
gdown.download(f"{BASE_URL}{GENAI_DATA_ID}{URL_SUFFIX}", os.path.join(DOWNLOAD_PATH, 'genai_data.zip'), quiet=False, fuzzy=True)

Downloading...
From (original): https://drive.google.com/uc?id=1yvGTLz4i4hapHEaW4cM8cvc1eTnTUMXc
From (redirected): https://drive.google.com/uc?id=1yvGTLz4i4hapHEaW4cM8cvc1eTnTUMXc&confirm=t&uuid=79edb14b-4b64-4937-995b-27055036660c
To: /home/developer/synth-data-proj/data/downloads/genai_data.zip
100%|██████████| 13.5G/13.5G [23:50<00:00, 9.42MB/s]


'data/downloads/genai_data.zip'

Unzip the files into the processed data directory.

In [57]:
# Zip file
zip_file = "data/downloads/genai_data.zip"
# Unzip the file into the processed data directory
with ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall("data/processed/")

Delete the zip files after their contents have been extracted.

## 3. Stage Data

Data is staged into a staging folder for the training process so modular chunks of data can be moved in and out as needed.

In [3]:
# Create the staging directory
os.makedirs(os.path.join('data', 'staging'), exist_ok=True)
os.makedirs(os.path.join('data', 'staging', 'train'), exist_ok=True)
os.makedirs(os.path.join('data', 'staging', 'val'), exist_ok=True)

In [4]:
# Get the list of training data csv files
trn_csv_files = {
    "3d_model": "data/processed/3d_models/train_data_splits.csv",
    "baseline": "data/processed/baseline/train_data_splits.csv",
    "clipart": "data/processed/clipart/train_data_splits.csv",
    "genai": "data/processed/genai/train_data_splits.csv"
}

In [5]:
# Read the files into a dataframe
t_model_3d_splits = pd.read_csv(trn_csv_files['3d_model'])
t_baseline_splits = pd.read_csv(trn_csv_files['baseline'])
t_clipart_splits = pd.read_csv(trn_csv_files['clipart'])
t_genai_splits = pd.read_csv(trn_csv_files['genai'])

In [6]:
# Get the list of validation data csv files
val_csv_files = {
    "3d_model": "data/processed/3d_models/val_data_splits.csv",
    "baseline": "data/processed/baseline/val_data_splits.csv",
    "clipart": "data/processed/clipart/val_data_splits.csv",
    "genai": "data/processed/genai/val_data_splits.csv"
}

In [7]:
# Read the files into a dataframe
v_model_3d_splits = pd.read_csv(val_csv_files['3d_model'])
v_baseline_splits = pd.read_csv(val_csv_files['baseline'])
v_clipart_splits = pd.read_csv(val_csv_files['clipart'])
v_genai_splits = pd.read_csv(val_csv_files['genai'])

In [8]:
def gen_data_mix(baseline_pct, modeled_pct, clipart_pct, genai_pct, stage='train', seed=42):
    """Generate a mixed dataset based on the specified percentages."""
    if stage == 'train':
        baseline_data = t_baseline_splits.sample(frac=baseline_pct, random_state=seed*10)
        model_3d_data = t_model_3d_splits.sample(frac=modeled_pct, random_state=seed*100)
        clipart_data = t_clipart_splits.sample(frac=clipart_pct, random_state=seed*1000)
        genai_data = t_genai_splits.sample(frac=genai_pct, random_state=seed*10000)
    else:
        baseline_data = v_baseline_splits.sample(frac=baseline_pct, random_state=seed*13)
        model_3d_data = v_model_3d_splits.sample(frac=modeled_pct, random_state=seed*133)
        clipart_data = v_clipart_splits.sample(frac=clipart_pct, random_state=seed*1333)
        genai_data = v_genai_splits.sample(frac=genai_pct, random_state=seed*13333)

    mixed_data = pd.concat([baseline_data, model_3d_data, clipart_data, genai_data]).reset_index(drop=True)
    return mixed_data

In [9]:
# For each file in the data series, copy it to the staging folder
def copy_to_staging(df, stage='train'):
    """Copy files listed in the dataframe to the staging directory."""
    for idx, row in df.iterrows():
        src_image_path = row['img_path']
        src_label_path = row['ann_path']
        
        # Convert paths from windows to unix if necessary
        src_image_path = src_image_path.replace('\\', '/')
        src_label_path = src_label_path.replace('\\', '/')

        # Define destination paths
        dest_image_path = os.path.join('data', 'staging', stage, os.path.basename(src_image_path))
        dest_label_path = os.path.join('data', 'staging', stage, os.path.basename(src_label_path))

        # Copy image and label files to staging directory
        shutil.copy(src_image_path, dest_image_path)
        shutil.copy(src_label_path, dest_label_path)

In [10]:
# Cleanup - delete all files in the staging folders
def cleanup_staging():
    """Delete all files in the staging directories."""
    staging_dirs = [os.path.join('data', 'staging', 'train'), os.path.join('data', 'staging', 'val')]
    for staging_dir in staging_dirs:
        for filename in os.listdir(staging_dir):
            file_path = os.path.join(staging_dir, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print(f'Failed to delete {file_path}. Reason: {e}')

In [70]:
# Create the yaml file for YOLO model training
# Create filename and file path
yaml_filename = 'detect.yaml'
yaml_path = os.path.join('data', 'staging', yaml_filename)

# YAML content
yaml_content = """
path: data/staging  # dataset root dir (leave empty for HUB)
train: train  # train images (relative to 'path')
val:   val    # val images (relative to 'path')

names:
  0: drone
"""

# Write the YAML content to the file
with open(yaml_path, 'w') as yaml_file:
    yaml_file.write(yaml_content)

## 4. Train Model

**Screening Experimental Design**

| **Run** | **3D Model Data** | **Clipart Data** | **Gen AI Data** |
| --- | --- | --- | --- |
| 1 | 0   | 100 | 0   |
| 2 | 100 | 0   | 0   |
| 3 | 0   | 50  | 50  |
| 4 | 50  | 0   | 50  |
| 5 | 0   | 0   | 0   |
| 6 | 50  | 50  | 0   |
| 7 | 0   | 0   | 100 |
| 8 | 33  | 33  | 33  |

In [71]:
# Prep data
cleanup_staging()

In [11]:
# Generate data mix dataframes
train_data = gen_data_mix(baseline_pct=0, modeled_pct=0, clipart_pct=1, genai_pct=0, stage='train', seed=111)
val_data = gen_data_mix(baseline_pct=0, modeled_pct=0, clipart_pct=1, genai_pct=0, stage='val', seed=111)

In [12]:
# Save train and val data dataframes to CSV for reference
train_data.to_csv(os.path.join('data', 'staging', 'train_data_0-0-100-0.csv'), index=False)
val_data.to_csv(os.path.join('data', 'staging', 'val_data_0-0-100-0.csv'), index=False)

In [75]:
# Copy files to staging directories
copy_to_staging(train_data, stage='train')
copy_to_staging(val_data, stage='val')

In [76]:
# Training config
config = {
    "lr0": 0.00099,
    "lrf": 0.01081,
    "momentum": 0.85521,
    "weight_decay": 0.00052,
    "hsv_h": 0.01634,
    "hsv_s": 0.7638,
    "hsv_v": 0.38363,
    "translate": 0.07243,
    "fliplr": 0.5,
    "mosaic": 0.95755
}

In [77]:
# Load the pre-trained YOLO model
model = YOLO("yolo11x.pt")

In [78]:
results = model.train(
    data='data/staging/detect.yaml',
    batch=0.85,
    epochs=200,
    patience=20,
    optimizer="AdamW",
    device=0,
    **config
)

New https://pypi.org/project/ultralytics/8.3.220 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.218 🚀 Python-3.11.0rc1 torch-2.9.0+cu128 CUDA:0 (NVIDIA H100 NVL, 95330MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=0.85, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/staging/detect.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.01634, hsv_s=0.7638, hsv_v=0.38363, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.00099, lrf=0.01081, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11x.pt, momentum=0.85521, mosaic=0.95755, multi_scale=False, name=train, nbs